# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Importing all the modules needed for the project
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Following will import the API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = os.path.join('..', 'WeatherPy', 'Output', 'city_weather.csv')

city_df = pd.read_csv(csv_file)
city_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,5848280,Kapaa,US,22.08,-159.32,77.00,65,75,6.93
1,4020109,Atuona,PF,-9.80,-139.03,82.45,75,84,11.34
2,2194098,Ahipara,NZ,-35.17,173.17,65.34,66,36,6.89
3,3833367,Ushuaia,AR,-54.80,-68.30,41.00,81,20,5.82
4,3985710,Cabo San Lucas,MX,22.89,-109.91,68.47,60,75,10.29
...,...,...,...,...,...,...,...,...,...
559,2698729,Kungsbacka,SE,57.49,12.08,31.32,86,8,6.93
560,2013727,Vanavara,RU,60.34,102.28,35.65,73,98,9.53
561,2037886,Dandong,CN,40.13,124.39,60.60,27,0,6.89
562,1488429,Uray,RU,60.12,64.78,38.01,86,87,11.30


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = city_df[(city_df['Temperature'] >= 75) & (city_df['Temperature'] <= 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Clouds'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
8,6355222,Yulara,AU,-25.24,130.99,82.40,18,9,4.70
12,4036284,Alofi,NU,-19.06,-169.92,82.40,65,9,5.82
15,2244991,Tambacounda,SN,13.75,-13.76,78.04,39,0,8.12
31,1254046,Tura,IN,25.52,90.22,87.40,24,0,5.99
53,1185095,Teknāf,BD,20.86,92.31,86.23,57,6,3.02
118,1637001,Biak,ID,-0.91,122.88,86.00,60,4,4.63
123,2450173,Taoudenni,ML,22.68,-3.98,80.58,14,0,6.33
186,1273856,Colgong,IN,25.27,87.22,89.73,21,0,1.90
202,1255175,Talāja,IN,21.35,72.05,86.85,35,0,7.45
246,1282027,Malé,MV,4.17,73.51,84.36,67,10,2.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name'] = ""
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
8,6355222,Yulara,AU,-25.24,130.99,82.40,18,9,4.70,
12,4036284,Alofi,NU,-19.06,-169.92,82.40,65,9,5.82,
15,2244991,Tambacounda,SN,13.75,-13.76,78.04,39,0,8.12,
31,1254046,Tura,IN,25.52,90.22,87.40,24,0,5.99,
53,1185095,Teknāf,BD,20.86,92.31,86.23,57,6,3.02,
118,1637001,Biak,ID,-0.91,122.88,86.00,60,4,4.63,
123,2450173,Taoudenni,ML,22.68,-3.98,80.58,14,0,6.33,
186,1273856,Colgong,IN,25.27,87.22,89.73,21,0,1.90,
202,1255175,Talāja,IN,21.35,72.05,86.85,35,0,7.45,
246,1282027,Malé,MV,4.17,73.51,84.36,67,10,2.75,


In [7]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df        

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
8,6355222,Yulara,AU,-25.24,130.99,82.40,18,9,4.70,Desert Gardens Hotel - Ayers Rock Resort
12,4036284,Alofi,NU,-19.06,-169.92,82.40,65,9,5.82,Taloa Heights
15,2244991,Tambacounda,SN,13.75,-13.76,78.04,39,0,8.12,NaN
31,1254046,Tura,IN,25.52,90.22,87.40,24,0,5.99,Hotel Polo Orchid
53,1185095,Teknāf,BD,20.86,92.31,86.23,57,6,3.02,Milky Resort
118,1637001,Biak,ID,-0.91,122.88,86.00,60,4,4.63,NaN
123,2450173,Taoudenni,ML,22.68,-3.98,80.58,14,0,6.33,NaN
186,1273856,Colgong,IN,25.27,87.22,89.73,21,0,1.90,SPOT ON 61925 Hotel Prerna
202,1255175,Talāja,IN,21.35,72.05,86.85,35,0,7.45,Velvettas Resorts
246,1282027,Malé,MV,4.17,73.51,84.36,67,10,2.75,Season Holidays


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))